In [24]:
%%bash

nvidia-smi

Fri Oct  7 00:19:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    27W /  70W |   1826MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [25]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-1idrwh09/farm-haystack_0352c049e1fa4d8cbc08fc2cbad88364
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-1idrwh09/farm-haystack_0352c049e1fa4d8cbc08fc2cbad88364
  Resolved https://github.com/deepset-ai/haystack.git to commit b84a6b17165dbf10665794b5decdefa113300749
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [26]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [27]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [28]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/tutorial3"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is: {"name": "<some-document-name>", "content": "<the-actual-text>"}

# Let's have a look at the first 3 entries:
print(docs[:3])

# Now, let's write the docs to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.import_utils:Found data stored in 'data/tutorial3'. Delete this first if you really want to fetch new data.
INFO:haystack.utils.preprocessing:Converting data/tutorial3/198_A_Clash_of_Kings.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/340_Roose_Bolton.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/425_No_One__Game_of_Thrones_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/228_Souad_Faress.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/469_Outline_of_A_Song_of_Ice_and_Fire_franchise.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/508_A_Game_of_Thrones__Second_Edition__card_game_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/401_Power_Is_Power.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/232_Tommen_Baratheon.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial3/233_Myrcella_Baratheon.txt
INFO:haystack.utils.preprocessing:Converting data/t

[<Document: {'content': "'''''A Clash of Kings''''' is the second novel in ''A Song of Ice and Fire'', an epic fantasy series by American author George R. R. Martin expected to consist of seven volumes. It was first published on 16 November 1998 in the United Kingdom, although the first United States edition did not follow until February 2, 1999 Like its predecessor, ''A Game of Thrones'', it won the Locus Award (in 1999) for Best Novel and was nominated for the Nebula Award (also in 1999) for best novel. In May 2005 Meisha Merlin released a limited edition of the novel, fully illustrated by John Howe.\nThe novel has been adapted for television by HBO as the second season of the TV series ''Game of Thrones''.\n''A Clash of Kings'' is also the name of the first expansion to the ''Game of Thrones'' board game.", 'content_type': 'text', 'score': None, 'meta': {'name': '198_A_Clash_of_Kings.txt'}, 'embedding': None, 'id': '93db7b2efade3240851b5e6fffb1eb07'}>, <Document: {'content': "\n== P

INFO:haystack.document_stores.base:Duplicate Documents: Document with id '5d79fbf801011475553a09ab068f02e2' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '9e9a3181b6bc168b4a25429b641e8c86' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '817c9fdc70cdd9a74075ff1a32503409' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '4fe5d3396a52edff7367d6137a7ea654' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '7ed12f389f7f085bb30c7d00abd26f81' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '7ed12f389f7f085bb30c7d00abd26f81' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '9e9a3181b6bc168b4a25429b641e8c86' already exists in index 'document'

In [29]:
# An in-memory TfidfRetriever based on Pandas dataframes
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

INFO:haystack.nodes.retriever.sparse:Found 2357 candidate paragraphs from 2357 docs in DB


In [30]:

from haystack.nodes import FARMReader


# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [31]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [32]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k for retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.88s/ Batches]


In [33]:
from pprint import pprint

pprint(prediction)


{'answers': [<Answer {'answer': 'Eddard', 'type': 'extractive', 'score': 0.9919580221176147, 'context': "s Nymeria after a legendary warrior queen. She travels with her father, Eddard, to King's Landing when he is made Hand of the King. Before she leaves,", 'offsets_in_document': [{'start': 147, 'end': 153}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_id': 'ba2a8e87ddd95e380bec55983ee7d55f', 'meta': {'name': '43_Arya_Stark.txt'}}>,
             <Answer {'answer': 'Ned', 'type': 'extractive', 'score': 0.9767245054244995, 'context': "\n====Season 1====\nArya accompanies her father Ned and her sister Sansa to King's Landing. Before their departure, Arya's half-brother Jon Snow gifts A", 'offsets_in_document': [{'start': 46, 'end': 49}], 'offsets_in_context': [{'start': 46, 'end': 49}], 'document_id': '180c2a6b36369712b361a80842e79356', 'meta': {'name': '43_Arya_Stark.txt'}}>,
             <Answer {'answer': 'Robert Baratheon', 'type': 'extractive', 'score': 0.940885305404

In [34]:
from haystack.utils import print_answers


# Change `minimum` to `medium` or `all` to control the level of detail
print_answers(prediction, details="medium")


Query: Who is the father of Arya Stark?
Answers:
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,',
        'score': 0.9919580221176147},
    {   'answer': 'Ned',
        'context': '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's "
                   'half-brother Jon Snow gifts A',
        'score': 0.9767245054244995},
    {   'answer': 'Robert Baratheon',
        'context': 'hen Gendry gives it to Arya, he tells her he is the '
                   'bastard son of Robert Baratheon. Aware of their chances of '
                   'dying in the upcoming battle and Arya w',
        'score': 0.9408853054046631},
    {   'answer': 'Eddard and Catelyn Stark',
  